<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# Apilando redes recurrentes<a id="top"></a>

<i>Última actualización: 2024-03-07</small></i></div>
***

## Introducción

Las RNN apiladas (del inglés _stacked_) son un tipo de red neuronal formada por múltiples capas de RNN apiladas unas sobre otras.Se ha demostrado que las RNN apiladas mejoran el rendimiento de las RNN en diversas tareas al permitir que la red aprenda representaciones más complejas de datos secuenciales.

En una RNN apilada, la salida de una capa de la RNN se pasa como entrada a la capa siguiente, creando una representación jerárquica de la secuencia.

## Objetivos

En este cuaderno resolveremos un problema anterior con este tipo de arquitectura, viendo cómo implementar un modelo con varias capas recurrentes superpuestas.

## Bibliotecas y configuración

A continuación importaremos las librerías que se utilizarán a lo largo del cuaderno.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Descarga y preproceso de los datos

Una vez más, utilizaremos el conjunto de datos `mnist`.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255, x_test / 255

print(f'Training shape: {x_train.shape} input, {y_train.shape} output')
print(f'Test shape:     {x_test.shape} input, {y_test.shape} output')

## Modelo basado en múltiples capas de SRU

Una arquitectura multicapa de SRU es casi idéntica a una multicapa tradicional. Sin embargo, hay que tener en cuenta un concepto fundamental: **las capas recurrentes se alimentan con una secuencia de entradas, no sólo con una entrada**.

Para ello, haremos uso del argumento `return_sequences=True` de las unidades recurrentes. Al activarlo, hacemos que la red devuelva la secuencia completa de salidas, no sólo la última. Haciendo esto, la capa siguiente, que requiere una secuencia, será alimentada con una secuencia. Veamos la implementación utilizando dos capas `SimpleRNN` de 10 «unidades» cada una:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(units=8, return_sequences=True, input_shape=(28, 28)),
    tf.keras.layers.SimpleRNN(units=16, return_sequences=True),
    tf.keras.layers.SimpleRNN(units=10),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Podemos ver en el resumen que la salida de la primera capa (la que tiene `return_sequences=True`) devuelve una secuencia de 28 elementos (el tamaño de la secuencia) de dimensión 10 cada uno (su número de unidades). La segunda capa, que no hace explícito el argumento (que por defecto es `False`) devuelve sólo 10 valores, los correspondientes a la última salida de la unidad recurrente después de haberla alimentado con los 28 elementos de la secuencia.

Por último, vamos a compilar el modelo creado con la función de pérdida que corresponde a este tipo de problema con un optimizador de descenso de gradiente estocástico y vamos a añadir la métrica _accuracy_ (en realidad `sparse categorical accuracy`, esto es, una versión de la primera pero preparada para problemas de clasificación multiclase para salidas _sparse_) para ver cómo evoluciona este entrenamiento.

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy'],
)

### Entrenamiento del modelo

Por último, entrenaremos nuestra red durante $10$ epochs. Sí, si con las redes recurrentes era lento, con las redes apiladas lo es aún más.

In [ ]:
history = model.fit(x_train, y_train, epochs=25)

Veamos cómo ha ido el entrenamiento del modelo.

In [ ]:
pd.DataFrame(history.history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

Quizá la precisión con la que ha clasificado sea mejor, pero en este problema concreto, las redes convolucionales siguen siendo la mejor opción.

### Clasificación de nuevas muestras

Ahora imprimiremos algunos de los elementos de la prueba para ver cómo se han clasificado y cuáles han sido los errores.

In [ ]:
# Predict some of the test digits
ROWS, COLS = 5, 5
IMAGES = ROWS * COLS
ŷ_test = np.argmax(model.predict(x_test[:IMAGES], verbose=0), axis=1)
# And plot them
fig = plt.figure(figsize=(15, 15))
for i, (x, y, ŷ) in enumerate(zip(x_test[:IMAGES], y_test[:IMAGES], ŷ_test), 1):
    ax = fig.add_subplot(ROWS, COLS, i)
    ax.imshow(x, cmap='Greens' if y == ŷ else 'Reds')
    ax.set_title(f'Expected: {y}, predicted: {ŷ}')
    ax.grid(False)
plt.tight_layout()

## Conclusiones

Hemos mostrado cómo implementar una red neuronal recurrente apilada en Keras, en concreto un modelo RNN de dos capas con 10 unidades en cada capa. Y aunque lo hemos aplicado al problema `mnist`, hemos comprobado que no es la mejor opción (al menos en tiempo de entrenamiento).

Apilar RNN puede ser una técnica muy útil para modelar datos secuenciales, ya que permite al modelo capturar dependencias más complejas entre las entradas y las salidas.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>